In [2]:
import pandas as pd
import numpy as np

In [4]:
## reading brands json

file_path = 'brands.json'

df_brands = pd.read_json(file_path, lines=True)

## for id column
if '_id' in df_brands.columns and isinstance(df_brands.loc[0, '_id'], dict) and '$oid' in df_brands.loc[0, '_id']:
    df_brands['_id_$oid'] = df_brands['_id'].apply(lambda x: x['$oid'] if isinstance(x, dict) and '$oid' in x else None)
    df_brands = df_brands.drop(columns=['_id']) # Dropping the original nested column

## For 'cpg' column
if 'cpg' in df_brands.columns and isinstance(df_brands.loc[0, 'cpg'], dict):
    df_brands['cpg_$ref'] = df_brands['cpg'].apply(lambda x: x.get('$ref') if isinstance(x, dict) else None)
    df_brands['cpg_$id_$oid'] = df_brands['cpg'].apply(lambda x: x['$id']['$oid'] if isinstance(x, dict) and '$id' in x and isinstance(x['$id'], dict) and '$oid' in x['$id'] else None)
    df_brands = df_brands.drop(columns=['cpg']) # Dropping the original nested column

# Displaying the first few rows of the DataFrame
print("DataFrame head:")
df_brands.head()


DataFrame head:


,barcode,category,categoryCode,name,topBrand,brandCode,_id_$oid,cpg_$ref,cpg_$id_$oid
0,511111019862,Baking,BAKING,test brand @1612366101024,0.0,NaN,601ac115be37ce2ead437551,Cogs,601ac114be37ce2ead437550
1,511111519928,Beverages,BEVERAGES,Starbucks,0.0,STARBUCKS,601c5460be37ce2ead43755f,Cogs,5332f5fbe4b03c9a25efd0ba
2,511111819905,Baking,BAKING,test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176,601ac142be37ce2ead43755d,Cogs,601ac142be37ce2ead437559
3,511111519874,Baking,BAKING,test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051,601ac142be37ce2ead43755a,Cogs,601ac142be37ce2ead437559
4,511111319917,Candy & Sweets,CANDY_AND_SWEETS,test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827,601ac142be37ce2ead43755e,Cogs,5332fa12e4b03c9a25efd1e7


In [6]:
## reading users JSON

users_file_path = 'users.json'

# Read the JSON Lines file into a DataFrame
df_users = pd.read_json(users_file_path, lines=True)

# --- Flattening Nested Fields ---

# Flatten '_id' to '_id.$oid'
if '_id' in df_users.columns:
    df_users['_id_$oid'] = df_users['_id'].apply(lambda x: x['$oid'] if isinstance(x, dict) and '$oid' in x else None)
    df_users = df_users.drop(columns=['_id'])

# Convert date fields with '$date' to datetime objects
date_columns_users = ['createdDate', 'lastLogin']
for col in date_columns_users:
    if col in df_users.columns:
        # Extract the timestamp and convert to datetime, handling potential missing '$date'
        df_users[col] = df_users[col].apply(lambda x: pd.to_datetime(x['$date'], unit='ms') if isinstance(x, dict) and '$date' in x else None)

# Displaying the first few rows of the DataFrame
print("head of users dataframe:")
df_users.head()


head of users dataframe:


,active,createdDate,lastLogin,role,signUpSource,state,_id_$oid
0,True,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052
1,True,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052
2,True,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052
3,True,2021-01-03 15:25:30.554,2021-01-03 15:25:30.597,consumer,Email,WI,5ff1e1eacfcf6c399c274ae6
4,True,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052


In [8]:
## reading receipts JSON

receipts_file_path = 'receipts.json'

# Read the JSON Lines file into a DataFrame
df_receipts = pd.read_json(receipts_file_path, lines=True)

# --- Flattening Nested Fields ---

# Flatten '_id' to '_id_$oid'
if '_id' in df_receipts.columns:
    df_receipts['_id_$oid'] = df_receipts['_id'].apply(lambda x: x['$oid'] if isinstance(x, dict) and '$oid' in x else None)
    df_receipts = df_receipts.drop(columns=['_id'])

# Convert date fields with '$date' to datetime objects
date_columns = ['createDate', 'dateScanned', 'finishedDate', 'modifyDate', 'pointsAwardedDate', 'purchaseDate']
for col in date_columns:
    if col in df_receipts.columns:
        # Extract the timestamp and convert to datetime, handling potential missing '$date'
        df_receipts[col] = df_receipts[col].apply(lambda x: pd.to_datetime(x['$date'], unit='ms') if isinstance(x, dict) and '$date' in x else None)

df_exploded_items = df_receipts.explode('rewardsReceiptItemList')

# Normalize the nested dictionaries within the 'rewardsReceiptItemList' column
# This converts each item dictionary into its own set of columns.
df_normalized_item_details = pd.json_normalize(df_exploded_items['rewardsReceiptItemList'])

df_receipts = pd.concat([
    df_exploded_items.drop(columns=['rewardsReceiptItemList']).reset_index(drop=True),
    df_normalized_item_details.reset_index(drop=True)
], axis=1)

print("--- DataFrame for receipts.json ---")
print("First 5 rows for receipts dataset:")
df_receipts.head()

--- DataFrame for receipts.json ---
First 5 rows for receipts dataset:


,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,...,itemNumber,originalMetaBriteQuantityPurchased,pointsEarned,targetPrice,competitiveProduct,originalFinalPrice,originalMetaBriteItemPrice,deleted,priceAfterCoupon,metabriteCampaignId
0,500.0,"Receipt number 2 completed, bonus point schedu...",2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:36,2021-01-03 15:25:31,500.0,2021-01-03 00:00:00,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:48,2021-01-03 15:24:43,150.0,2021-01-02 15:24:43,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:48,2021-01-03 15:24:43,150.0,2021-01-02 15:24:43,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5.0,All-receipts receipt bonus,2021-01-03 15:25:37,2021-01-03 15:25:37,NaT,2021-01-03 15:25:42,NaT,5.0,2021-01-03 00:00:00,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5.0,All-receipts receipt bonus,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:39,2021-01-03 15:25:34,5.0,2021-01-03 00:00:00,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
df_brands.info()
print("\n")
print("______________________________________________________________________")
print("\n")
df_receipts.info()
print("\n")
print("______________________________________________________________________")
print("\n")
df_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1167 entries, 0 to 1166
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   barcode       1167 non-null   int64  
 1   category      1012 non-null   object 
 2   categoryCode  517 non-null    object 
 3   name          1167 non-null   object 
 4   topBrand      555 non-null    float64
 5   brandCode     933 non-null    object 
 6   _id_$oid      1167 non-null   object 
 7   cpg_$ref      1167 non-null   object 
 8   cpg_$id_$oid  1167 non-null   object 
dtypes: float64(1), int64(1), object(7)
memory usage: 82.2+ KB


______________________________________________________________________


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7381 entries, 0 to 7380
Data columns (total 48 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   bonusPointsEarned          

In [12]:
print("Number of unique barcodes in Brands")
print(df_brands["barcode"].nunique())

print("Number of unique barcodes in Receipts")
print(df_receipts["barcode"].nunique())

print("Number of unique user ids in Users")
print(df_users["_id_$oid"].nunique())

Number of unique barcodes in Brands
1160
Number of unique barcodes in Receipts
568
Number of unique user ids in Users
212


In [14]:

## question 1

unq_barc = df_receipts.groupby("rewardsGroup")['barcode'].nunique().nlargest(5)
print("Top 5 by receipts scanned by", unq_barc)
print("\n")
print("\n")
unq_barc = df_receipts.groupby("brandCode")['barcode'].nunique().nlargest(5)
print("Top 5 receipt scanned by", unq_barc)


Top 5 by receipts scanned by rewardsGroup
OSCAR MAYER LUNCH MEAT                            12
SARGENTO RICOTTA CHEESE                           11
ANNIE'S HOMEGROWN MULTI-SERVING MAC & CHEESE      10
SARGENTO NATURAL SHREDDED CHEESE 6OZ OR LARGER    10
SARGENTO STRING OR STICK CHEESE                    9
Name: barcode, dtype: int64




Top 5 receipt scanned by brandCode
HY-VEE           19
BRAND            18
KRAFT             9
HIDDEN VALLEY     7
KNORR             6
Name: barcode, dtype: int64


In [34]:
## question 2


df_receipts["monthScanned"] = df_receipts["dateScanned"].dt.to_period('M').dt.start_time

latest_month = df_receipts["monthScanned"].max()
scnd_latest_month = df_receipts["monthScanned"].max() - pd.DateOffset(months = 1)
thrd_latest_month = df_receipts["monthScanned"].max() - pd.DateOffset(months = 2)
frth_latest_month = df_receipts["monthScanned"].max() - pd.DateOffset(months = 3)

print("Top 5 receipts scanned in latest month for which we have rewards Group data")
print("\n")
print(df_receipts[df_receipts["monthScanned"]==latest_month].groupby("rewardsGroup")["totalSpent"].mean().nlargest(5))

print("___________________________________________________________________________________________________\n")

print("Top 5 receipts scanned in 2nd latest month for which we have rewards Group data")
print("\n")
print(df_receipts[df_receipts["monthScanned"]==scnd_latest_month].groupby("rewardsGroup")["totalSpent"].mean().nlargest(5))


print("___________________________________________________________________________________________________\n")

print("Top 5 receipts scanned in 3rd latest month for which we have rewards Group data")
print("\n")
print(df_receipts[df_receipts["monthScanned"]==thrd_latest_month].groupby("rewardsGroup")["totalSpent"].mean().nlargest(5))



print("___________________________________________________________________________________________________\n")

print("Top 5 receipts scanned in 4th latest month for which we have rewards Group data")
print("\n")
print(df_receipts[df_receipts["monthScanned"]==frth_latest_month].groupby("rewardsGroup")["totalSpent"].mean().nlargest(5))

Top 5 receipts scanned in latest month for which we have rewards Group data


Series([], Name: totalSpent, dtype: float64)
___________________________________________________________________________________________________

Top 5 receipts scanned in 2nd latest month for which we have rewards Group data


rewardsGroup
ANNIE'S HOMEGROWN MULTI-SERVING MAC & CHEESE      50.000
DUNCAN HINES CAKE MIX                             49.950
HEINZ GRAVY - JAR                                 49.950
SWISS MISS CAFÉ                                   25.000
SARGENTO NATURAL SHREDDED CHEESE 6OZ OR LARGER    17.125
Name: totalSpent, dtype: float64
___________________________________________________________________________________________________

Top 5 receipts scanned in 3rd latest month for which we have rewards Group data


rewardsGroup
KNORR BOUILLON                4721.950000
DUNCAN HINES FROSTING         4670.023333
DIET PEPSI 12 OZ 24+ COUNT    4644.060000
SIERRA MIST 12 OZ 12 PACK     4644.060000

In [16]:
## question 3

receipt_status = df_receipts.groupby("rewardsReceiptStatus")["totalSpent"].mean().reset_index()


print("Rejected ",receipt_status[receipt_status["rewardsReceiptStatus"]=="REJECTED"].iloc[:,1:])
print("___________________________________________________________________________________________________\n")
print("Finished/Accepted ",receipt_status[receipt_status["rewardsReceiptStatus"]=="FINISHED"])


Rejected     totalSpent
3    19.54497
___________________________________________________________________________________________________

Finished/Accepted    rewardsReceiptStatus   totalSpent
0             FINISHED  1244.372934
